In [0]:
import re
import argparse
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from collections import OrderedDict
from itertools import chain
import collections

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


building encoder


In [0]:
lr = 0.001
batch_size = 128
init = 1
dropout_prob= 0
decode_method=1
beam_width = 10
save_dir = 'drive/My Drive/data_PA3/'
num_epochs = 10
train_path = 'drive/My Drive/data_PA3/train.csv'
valid_path = 'drive/My Drive/data_PA3/valid.csv'
test_path = 'drive/My Drive/data_PA3/partial_test_400.csv'

In [0]:
train = pd.read_csv(train_path)
valid= pd.read_csv(valid_path)
test = pd.read_csv(test_path)

In [0]:
inembsize = 256
outembsize = 256

num_batches = len(train)//batch_size
encoder_layers = 1
decoder_layers = 2
state_size = 512

In [0]:
train_eng = train['ENG']
train_hin = train["HIN"]

valid_eng = valid["ENG"]
valid_hin = valid["HIN"]

In [0]:
def one_hot(batch_word, max_length, word_to_idx):
    vec_word = np.zeros((len(batch_word), max_length, len(word_to_idx)), dtype=float)
    for i, word in enumerate(batch_word):
        for j, char in enumerate(word):
            vec_word[i, j, char] = 1.
    return vec_word

In [0]:
def generate_text(prediction, batch_size, length, vocab_size, idx_to_word):

    batch_softmax = np.reshape(prediction, [batch_size, length, vocab_size])
    batch_word = []

    for word in batch_softmax:
        new_word = ''
        for char in word:
            vector_position = np.argmax(char)
            y_word = idx_to_word[vector_position]
            if y_word != 'PAD':
                new_word = new_word + y_word + ' '
            else:
                new_word = new_word + ''
        batch_word.append(new_word)

    return batch_word

In [0]:
## To remove later

# def variable_summaries(var):
#   """Attach a lot of summaries to a Tensor (for TensorBoard visualization)."""
#   with tf.name_scope('summaries'):
#     mean = tf.reduce_mean(var)
#     tf.summary.scalar('mean', mean)
#     with tf.name_scope('stddev'):
#       stddev = tf.sqrt(tf.reduce_mean(tf.square(var - mean)))
#     tf.summary.scalar('stddev', stddev)
#     tf.summary.scalar('max', tf.reduce_max(var))
#     tf.summary.scalar('min', tf.reduce_min(var))
#     tf.summary.histogram('histogram', var)

In [0]:
def load_data(train_eng, train_hin):
    
    # Feed input data in backwards for better translation performance.
    x_data = [(tf.compat.as_str(train_eng[_]).lower().split(' ')) for _ in range(len(train_eng))]
    x_data = [list(x_data[_]) for _ in range(len(x_data))]
    X_DATA_SIZE = len(x_data)

    y_data = [(tf.compat.as_str(train_hin[_]).lower().split(' ')) for _ in range(len(train_hin))]
    y_data = [list(y_data[_]) for _ in range(len(y_data))]
    Y_DATA_SIZE = len(y_data)


    x_word_list = []
    y_word_list = []
    [x_word_list.extend(x_data[_]) for _ in range(len(x_data))]
    [y_word_list.extend(y_data[_]) for _ in range(len(y_data))]

    x_word_dictionary = []
    y_word_dictionary = []
    x_word_dictionary.extend(collections.Counter(x_word_list).most_common(44))
    y_word_dictionary.extend(collections.Counter(y_word_list).most_common(84))
    

    x_idx_to_word = [word[0] for idx, word in enumerate(x_word_dictionary)]
    x_idx_to_word.insert(0, 'GO')
    x_idx_to_word.insert(1, 'EOS')
    x_idx_to_word.insert(2, 'PAD')
    x_idx_to_word.insert(3, 'UK')


    y_idx_to_word = [word[0] for idx, word in enumerate(y_word_dictionary)]
    y_idx_to_word.insert(0, 'GO')
    y_idx_to_word.insert(1, 'EOS')
    y_idx_to_word.insert(2, 'PAD')
    y_idx_to_word.insert(3, 'UK')

    x_word_to_idx = {word:ix for ix, word in enumerate(x_idx_to_word)}
    y_word_to_idx = {word: ix for ix, word in enumerate(y_idx_to_word)}

    X_VOCAB_SIZE = len(x_word_dictionary) + 4
    Y_VOCAB_SIZE = len(y_word_dictionary) + 4

    X_MAX_LENGTH = max([len(x_data[_]) for _ in range(len(x_data))])
    Y_MAX_LENGTH = max([len(y_data[_]) for _ in range(len(y_data))])

    return X_VOCAB_SIZE, Y_VOCAB_SIZE, x_idx_to_word, x_word_to_idx, y_idx_to_word, y_word_to_idx, X_MAX_LENGTH, Y_MAX_LENGTH, X_DATA_SIZE


In [0]:
X_VOCAB_SIZE, Y_VOCAB_SIZE, x_idx_to_word, x_word_to_idx, y_idx_to_word, y_word_to_idx, X_MAX_LENGTH, Y_MAX_LENGTH, X_DATA_SIZE = load_data(train_eng, train_hin)                                                                                                                   

X_VOCAB_SIZE_VALID, Y_VOCAB_SIZE_VALID, x_idx_to_word_VALID, x_word_to_idx_VALID, y_idx_to_word_VALID, y_word_to_idx_VALID, X_MAX_LENGTH_VALID, Y_MAX_LENGTH_VALID, X_DATA_SIZE_VALID = load_data(valid_eng, valid_hin)                                                                                                                   


In [0]:
def get_data(x_word_to_idx, y_word_to_idx, inputs = train_eng, labels = train_hin):


    x_data = inputs
    y_data = labels

    x_data = [(tf.compat.as_str(x_data[_]).lower().split(' ')) for _ in range(len(x_data))]
    x_data = [list(x_data[_]) for _ in range(len(x_data))]
       
    y_data = [(tf.compat.as_str(y_data[_]).lower().split(' ')) for _ in range(len(y_data))]
    y_data = [list(y_data[_]) for _ in range(len(y_data))]

    for i, word in enumerate(x_data):
        for j, charac in enumerate(word):
            if charac in x_word_to_idx:
                x_data[i][j] = x_word_to_idx[charac]
            else:
                x_data[i][j] = x_word_to_idx['UK']

    for i, word in enumerate(y_data):
        for j, charac in enumerate(word):
            if charac in y_word_to_idx:
                y_data[i][j] = y_word_to_idx[charac]
            else:
                y_data[i][j] = y_word_to_idx['UK']
                
    return x_data, y_data

In [0]:
x_data , y_data = get_data( x_word_to_idx, y_word_to_idx, inputs = train_eng, labels = train_hin)                                              

In [0]:
x_data_val , y_data_val = get_data(x_word_to_idx, y_word_to_idx, inputs = valid_eng, labels = valid_hin)

In [0]:
def encoding(encoder_x):
  
      word_embeddings = tf.get_variable('encoder_word_embeddings', [X_VOCAB_SIZE, inembsize])
      encoder_embedded_word_ids = tf.nn.embedding_lookup(word_embeddings, encoder_x)
      encoder_embedded_word_ids = tf.reshape(encoder_embedded_word_ids, [-1, X_MAX_LENGTH, inembsize])
      
      for n in range(encoder_layers): 
                encoder_cell = tf.nn.rnn_cell.LSTMCell(state_size, state_is_tuple=True)
                encoder_out, encoder_state = tf.nn.bidirectional_dynamic_rnn(
                cell_fw = encoder_cell,  ## initializer, reuse to be set
                cell_bw = encoder_cell,  ## initializer, reuse to be set
                inputs = encoder_embedded_word_ids,
                dtype = tf.float32)
            
      return encoder_state

In [0]:
def decoding_train(decoder_embedded_word_ids, decoder_cell, encoder_state, dense_layer, train_sequence_length):
  
      training_helper = tf.contrib.seq2seq.TrainingHelper(inputs = decoder_embedded_word_ids, sequence_length = train_sequence_length)

      training_decoder = tf.contrib.seq2seq.BasicDecoder(
              cell = decoder_cell,
              helper = training_helper,
              initial_state = encoder_state,
              output_layer = dense_layer)

      training_decoder_output, _, _ = tf.contrib.seq2seq.dynamic_decode(
              decoder = training_decoder,
              impute_finished = True,   ## impute finished make sure that weights causing correctly predicted words are not changed further
              maximum_iterations = Y_MAX_LENGTH)   
      
      training_logits = training_decoder_output.rnn_output
      training_logits_max = training_decoder_output.sample_id
      
      return training_logits, training_logits_max

In [0]:
def decoding_infer(word_embeddings, decoder_cell, encoder_state, dense_layer):
     
      #decoder for test data
          predicting_helper = tf.contrib.seq2seq.GreedyEmbeddingHelper(embedding = word_embeddings,
                  start_tokens = tf.fill([batch_size], y_word_to_idx['GO']), end_token = y_word_to_idx['EOS'])

          predicting_decoder = tf.contrib.seq2seq.BasicDecoder(
                  cell = decoder_cell,
                  helper = predicting_helper,
                  initial_state = encoder_state,
                  output_layer = dense_layer)

          predicting_decoder_output, _, _ = tf.contrib.seq2seq.dynamic_decode(
                  decoder = predicting_decoder,
                  impute_finished = True,
                  maximum_iterations = Y_MAX_LENGTH)
          
          
          predicting_ids = predicting_decoder_output.sample_id
          
          return predicting_ids

In [0]:
def decoding(decoder_x, encoder_state, train_sequence_length):
      word_embeddings = tf.get_variable('decoder_word_embeddings', [Y_VOCAB_SIZE, outembsize])
      decoder_embedded_word_ids = tf.nn.embedding_lookup(word_embeddings, decoder_x)
      decoder_embedded_word_ids = tf.reshape(decoder_embedded_word_ids,  [-1, Y_MAX_LENGTH,  outembsize])
      
      decoder_cell = tf.nn.rnn_cell.MultiRNNCell([tf.nn.rnn_cell.LSTMCell(state_size) for _ in range(decoder_layers)])
      dense_layer = tf.layers.Dense(Y_VOCAB_SIZE)
      
      with tf.variable_scope("decode"):
            train_output, train_output_max = decoding_train(decoder_embedded_word_ids, decoder_cell, encoder_state, dense_layer, train_sequence_length)
          
      
      with tf.variable_scope("decode", reuse=True):       
            infer_output = decoding_infer(word_embeddings, decoder_cell, encoder_state, dense_layer)
        
      return train_output,  train_output_max, infer_output

In [0]:
def get_accuracy(logits, labels):
  
    max_seq = Y_MAX_LENGTH
    
    for i in range(len(logits)):
       if len(logits[i])<max_seq:
          logits[i] = np.pad(
            logits[i],
            [(0,0),(0,max_seq - len(logits[i]))],
            'constant')
          
    for i in range(len(labels)):
       if len(labels[i])<max_seq:
          labels[i] = np.pad(
            labels[i],
            [(0,0),(0,max_seq - len(labels[i]))],
            'constant')

    return np.mean(np.equal(logits, labels))

In [0]:
def pad(sentence_batch, pad_int, max_sentence):
    """Pad sentences with <PAD> so that each sentence of a batch has the same length"""
    
    return [sentence + [pad_int] * (max_sentence - len(sentence)) for sentence in sentence_batch]

In [0]:
def get_batches(x_data, y_data, batch_size, x_pad_int, y_pad_int):
    """Batch targets, sources, and the lengths of their sentences together"""
    for batch_i in range(0, len(x_data)//batch_size):
        start_i = batch_i * batch_size

        # Slice the right amount for the batch
        x_data_batch = x_data[start_i:start_i + batch_size]
        y_data_batch = y_data[start_i:start_i + batch_size]

        # Pad
        pad_x_batch = np.array(pad(x_data_batch, x_pad_int, X_MAX_LENGTH))
        pad_y_batch = np.array(pad(y_data_batch, y_pad_int, Y_MAX_LENGTH))

        # Need the lengths for the _lengths parameters
        pad_y_lengths = []
        for y in pad_y_batch:
            pad_y_lengths.append(len(y))

        pad_x_lengths = []
        for x in pad_x_batch:
            pad_x_lengths.append(len(x))

        yield pad_x_batch, pad_y_batch, pad_x_lengths, pad_y_lengths

In [0]:
tf_graph = tf.Graph()
with tf_graph.as_default():
  
      encoder_x = tf.placeholder(dtype=tf.int32, shape=[None, None]) 
      decoder_x = tf.placeholder(dtype=tf.int32, shape=[None, None]) 
      #y = tf.placeholder(dtype=tf.float32, shape=[None, None, None])
      y_sequence_length =  tf.placeholder(dtype=tf.int32, shape=[None]) 
      
      encoder_state=  encoding(encoder_x)
      train_logits, train_logits_max,  infer_output = decoding(decoder_x, encoder_state, y_sequence_length)
      
      
      masks = tf.sequence_mask(y_sequence_length, Y_MAX_LENGTH, dtype=tf.float32)
      
      softmax_out = tf.nn.softmax(train_logits)
      
      #cost =  tf.nn.softmax_cross_entropy_with_logits(logits=train_logits, labels=y)
      cost = tf.contrib.seq2seq.sequence_loss(
            train_logits,
            decoder_x,
            masks)
      

      total_cost = tf.reduce_mean(cost)
      optimizer = tf.train.AdamOptimizer(lr).minimize(total_cost)
      
    
     # accuracy = get_accuracy(train_logits, decoder_x)
      
#       y_t = tf.argmax(softmax_out,axis=2)
#       y_t = tf.cast(y_t, tf.int32)

#       prediction = y_t
#       mask_label = decoder_x

#       correct_pred = tf.equal(prediction, mask_label)
#       accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

In [60]:
with tf.Session(graph=tf_graph) as sess:

    merged = tf.summary.merge_all()
    saver = tf.train.Saver(max_to_keep=30)
    
    batch_x_val, batch_y_val, batch_x_sequence_length_val, batch_y_sequence_length_val = next(get_batches(x_data_val, y_data_val, batch_size, 2,2))
 
    sess.run(tf.global_variables_initializer())

    epoch = 0

    while epoch< num_epochs: 
        epoch_loss = 0
        epoch_loss_val =0
        epoch_acc_agg =0
        epoch_acc_agg_val =0
        n_steps=0
        
        for batch, (batch_x, batch_y, batch_x_sequence_length, batch_y_sequence_length) in enumerate(get_batches(x_data, y_data, batch_size, 2,2)):
 
#             start_batch = batch * batch_size
#             end_batch = start_batch + batch_size

#             batch_loss = 0
#             batch_y_sequence_length=[]
            
#             batch_x = x_data[start_batch:end_batch]
#             batch_y = y_data[start_batch:end_batch]
            
#             for y in batch_y:
#                 batch_y_sequence_length.append(len(y))
            
#             batch_y_sequence_length = np.array(batch_y_sequence_length)
            
#             for _ in range(len(batch_x)):
#                 x_length = len(batch_x[_])
#                 y_length = len(batch_y[_])

#                 batch_x[_].extend(np.zeros([X_MAX_LENGTH - x_length], dtype=int))
#                 batch_y[_].extend(np.zeros([Y_MAX_LENGTH - y_length], dtype=int))
            
   
#             batch_x_valid = x_data_val[start_batch:end_batch]
#             batch_y_valid = y_data_val[start_batch:end_batch]
        
#             y_one_hot = one_hot(batch_y, Y_MAX_LENGTH, y_word_to_idx)
#             y_one_hot_valid = one_hot(batch_y_valid, Y_MAX_LENGTH, y_word_to_idx)
            
  
            
            opt = sess.run(optimizer, feed_dict={encoder_x:batch_x,  decoder_x:batch_y,   y_sequence_length: batch_y_sequence_length })
            batch_loss  = sess.run([total_cost], feed_dict={encoder_x: batch_x,  decoder_x: batch_y,   y_sequence_length: batch_y_sequence_length })     
            logits, logits_max = sess.run([train_logits, train_logits_max], feed_dict={encoder_x: batch_x,  decoder_x: batch_y,   y_sequence_length: batch_y_sequence_length })    
            #_logits = np.argmax(logits, axis=2)
            
            accuracy = get_accuracy(logits_max, batch_y)
            #print(accuracy)
            
            
            
            #opt = sess.run(optimizer, feed_dict={encoder_x: x_data_valid, decoder_x: y_data_valid, y: y_one_hot_valid})
            batch_loss_val = sess.run([total_cost], feed_dict={encoder_x:  batch_x_val, decoder_x:  batch_y_val, y_sequence_length: batch_y_sequence_length_val })     
            logits_val = sess.run([infer_output], feed_dict={encoder_x:  batch_x_val, decoder_x:  batch_y_val, y_sequence_length: batch_y_sequence_length_val })
            #print(np.shape(logits_val))
            
            accuracy_val = get_accuracy(logits_val, batch_y_val)
            
            if batch % 10 == 0:

#                    save_output_path = save_dir + 'TRAINING_TRANSLATION_OUTPUT_EPOCH_{}_STATE-SIZE_{}_NUM-LAYERS_{}_LEARNING-RATE{}_EMBEDDING-SIZE_{}.txt'.format(epoch, state_size, decoder_layers, lr, inembsize)
                      text = lstm_support.generate_text(_prediction_series, batch_size, Y_MAX_LENGTH, Y_VOCAB_SIZE, y_idx_to_word)
  
                      print('Batch {}/{} Training Loss: {}, Training Accuracy: {} '.format(batch+1, num_batches, batch_loss[0], accuracy))
                      print('Batch {}/{} Validation Loss: {}, Validation Accuracy: {} '.format(batch+1, num_batches, batch_loss_val[0], accuracy_val))

            epoch_loss += batch_loss[0]
            epoch_loss_val += batch_loss_val[0]
            epoch_acc_agg += accuracy
            epoch_acc_agg_val += accuracy_val
            n_steps+=1
            
        print('Epoch {} Training Loss: {}, Training Accuracy: {} '.format(epoch+1, epoch_loss, epoch_acc_agg/n_steps))
        print('Epoch {} Validation Loss: {}, Training Accuracy: {} '.format(epoch+1, epoch_loss_val, epoch_acc_agg_val/n_steps))
        
        if epoch == 0:
            path = saver.save(sess, save_dir, epoch)
        else:
            path = saver.save(sess, save_dir, epoch, write_meta_graph=False)

        print("Saved in: %s" % path)
        epoch+=1

Batch 1/102 Training Loss: 3.299989700317383, Training Accuracy: 0.8375756048387096 
Batch 1/102 Validation Loss: 3.1928985118865967, Validation Accuracy: 0.8949092741935484 
Batch 11/102 Training Loss: 0.701086699962616, Training Accuracy: 0.8602570564516129 
Batch 11/102 Validation Loss: 0.4752120077610016, Validation Accuracy: 0.8961693548387096 
Batch 21/102 Training Loss: 0.6003575325012207, Training Accuracy: 0.8612651209677419 
Batch 21/102 Validation Loss: 0.44034773111343384, Validation Accuracy: 0.8989415322580645 
Batch 31/102 Training Loss: 0.5613174438476562, Training Accuracy: 0.859375 
Batch 31/102 Validation Loss: 0.41438376903533936, Validation Accuracy: 0.8971774193548387 
Batch 41/102 Training Loss: 0.48603853583335876, Training Accuracy: 0.8731098790322581 
Batch 41/102 Validation Loss: 0.37895667552948, Validation Accuracy: 0.8931451612903226 
Batch 51/102 Training Loss: 0.49619928002357483, Training Accuracy: 0.8762600806451613 
Batch 51/102 Validation Loss: 0.368

KeyboardInterrupt: ignored

In [0]:
# tf_graph = tf.Graph()
# with tf_graph.as_default():
  
#   encoder_x = tf.placeholder(dtype=tf.int32, shape=[None, None]) #[batch_size, X_MAX_LENGTH]
#   decoder_x = tf.placeholder(dtype=tf.int32, shape=[None, None]) #[batch_size, Y_MAX_LENGTH, Y_VOCAB_SIZE]
#   y = tf.placeholder(dtype=tf.float32, shape=[None, None, None])#[batch_size, Y_MAX_LENGTH, Y_VOCAB_SIZE]
  
#   with tf.variable_scope('encoder_word_embeddings'):

#       word_embeddings = tf.get_variable('encoder_word_embeddings', [X_VOCAB_SIZE, inembsize])
#       encoder_embedded_word_ids = tf.nn.embedding_lookup(word_embeddings, encoder_x)
#       encoder_embedded_word_ids = tf.reshape(encoder_embedded_word_ids, [-1, X_MAX_LENGTH, inembsize])

#   with tf.variable_scope('encoder'):

#       for n in range(encoder_layers): 
#                 encoder_cell = tf.nn.rnn_cell.LSTMCell(state_size, state_is_tuple=True)
#                 encoder_out, encoder_state = tf.nn.bidirectional_dynamic_rnn(
#                 cell_fw = encoder_cell,  ## initializer, reuse to be set
#                 cell_bw = encoder_cell,  ## initializer, reuse to be set
#                 inputs = encoder_embedded_word_ids,
#                 dtype = tf.float32)

#   #           (out_fw, out_bw), (state_fw, state_bw) = tf.nn.bidirectional_dynamic_rnn(
#   #               cell_fw = tf.nn.rnn_cell.LSTMCell(state_size//2, state_is_tuple=True),  ## initializer, reuse to be set
#   #               cell_bw = tf.nn.rnn_cell.LSTMCell(state_size//2, state_is_tuple=True),  ## initializer, reuse to be set
#   #               inputs = encoder_embedded_word_ids,

#   #               sequence_length =  [61]* batch_size,
#   #               dtype = tf.float32)

#   #           encoder_embedded_word_ids = tf.concat((out_fw, out_bw), 2)

#   #           bi_state_c = tf.concat((state_fw.c, state_bw.c), -1)
#   #           bi_state_h = tf.concat((state_fw.h, state_bw.h), -1)
#   #           bi_lstm_state = tf.nn.rnn_cell.LSTMStateTuple(c=bi_state_c, h=bi_state_h)
#   #           encoder_state = tuple([bi_lstm_state] * encoder_layers)

#   #           encoder_state = tuple(encoder_state[-1] for _ in range(encoder_layers))


#   with tf.variable_scope('decoder_word_embeddings'):

#       word_embeddings = tf.get_variable('decoder_word_embeddings', [Y_VOCAB_SIZE, outembsize])
#       decoder_embedded_word_ids = tf.nn.embedding_lookup(word_embeddings, decoder_x)
#       decoder_embedded_word_ids = tf.reshape(decoder_embedded_word_ids,  [-1, Y_MAX_LENGTH,  outembsize])



#   with tf.variable_scope('decoder'):

#       decoder_cell = tf.nn.rnn_cell.MultiRNNCell([tf.nn.rnn_cell.LSTMCell(state_size) for _ in range(decoder_layers)])
#       dense_layer = tf.layers.Dense(Y_VOCAB_SIZE)

#       #decoder for training data
#       training_helper = tf.contrib.seq2seq.TrainingHelper(inputs = decoder_embedded_word_ids, sequence_length = [62]* batch_size)



#       training_decoder = tf.contrib.seq2seq.BasicDecoder(
#               cell = decoder_cell,
#               helper = training_helper,
#               initial_state = encoder_state,
#               output_layer = dense_layer)

#       training_decoder_output, _, _ = tf.contrib.seq2seq.dynamic_decode(
#               decoder = training_decoder,
#               impute_finished = True,   ## impute finished make sure that weights causing correctly predicted words are not changed further
#               maximum_iterations = Y_MAX_LENGTH)   
      
#       training_logits = training_decoder_output.rnn_output
#       softmax_out = tf.nn.softmax(training_logits)
      
#       cost =  tf.nn.softmax_cross_entropy_with_logits(logits=training_logits, labels=y)

#       total_cost = tf.reduce_mean(cost)
#       optimizer = tf.train.AdamOptimizer(lr).minimize(total_cost)

#       y_t = tf.argmax(softmax_out,axis=2)
#       y_t = tf.cast(y_t, tf.int32)

#       #prediction = tf.boolean_mask(y_t, masks)
#       prediction = y_t
#       #mask_label = tf.boolean_mask(decoder_x, masks)
#       mask_label = decoder_x

#       correct_pred = tf.equal(prediction, mask_label)
#       accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
           
#   with tf.variable_scope('decoder', reuse=True):   
#       #decoder for test data
#       predicting_helper = tf.contrib.seq2seq.GreedyEmbeddingHelper(embedding = word_embeddings,
#               start_tokens = tf.fill([batch_size], y_word_to_idx['GO']), end_token = y_word_to_idx['EOS'])
                           
#       predicting_decoder = tf.contrib.seq2seq.BasicDecoder(
#               cell = decoder_cell,
#               helper = predicting_helper,
#               initial_state = encoder_state,
#               output_layer = dense_layer)

#       predicting_decoder_output, _, _ = tf.contrib.seq2seq.dynamic_decode(
#               decoder = predicting_decoder,
#               impute_finished = True,
#               maximum_iterations = 2 * (X_MAX_LENGTH))

#       predicting_ids = predicting_decoder_output.sample_id
      
#       #predicting_ids = tf.cast(predicting_ids, tf.float32)
#       #softmax_out_val = tf.nn.softmax(predicting_ids)
# #       print(predicting_ids)
      
# #       cost_val =  tf.nn.softmax_cross_entropy_with_logits(logits=predicting_ids, labels=y)

# #       total_cost_val = tf.reduce_mean(cost_val)
#       #optimizer = tf.train.AdamOptimizer(lr).minimize(total_cost_val)
      
#       #y_t_val = tf.argmax(predicting_ids,axis=2)
#       #y_t_val = tf.cast(y_t_val, tf.int32)

#       #prediction = tf.boolean_mask(y_t, masks)
# #       prediction_val = predicting_ids
      
# #       prediction_val = tf.cast(prediction_val, tf.int32)
# #       #mask_label = tf.boolean_mask(decoder_x, masks)
      
# #       mask_label_val = decoder_x

# #       correct_pred_val= tf.equal(prediction_val, mask_label_val)
# #       accuracy_val = tf.reduce_mean(tf.cast(correct_pred_val, tf.float32))
      
      
#       #masks = tf.sequence_mask(tf.count_nonzero(decoder_x, 1, dtype=tf.int32), Y_MAX_LENGTH, dtype=tf.float32)  

#       #cost = tf.contrib.seq2seq.sequence_loss(logits = softmax_out,targets = decoder_x)

      
